# Library Imports

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import ttest_rel
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Importing Data

Enable displaying all columns when we print dataframes.

In [2]:
# Display all columns in output
pd.set_option('display.max_columns', None)

Import training data. Change path if needed.

In [3]:
TRAIN_PATH = './petfinder-adoption-prediction/train.csv'
df = pd.read_csv(TRAIN_PATH)

# General Pre-Processing

These are the pre-processing steps that the whole team followed:
- Drop unnecessary columns:
  - `Name` was dropped because it has many missing values.
  - `RescuerID` and `PetID` were dropped because they provide no meaning to our models.
  - `Description` was dropped because we are not intending to do sentiment analysis on text for our project.
- Drop rows with missing data.
- Remove outliers for `Fee` and `Age` (numerical values). Outliers are those at least 5 standard deviations above the average for a column.

I got help from Ashley, Ryan, and Charles for the code for these preprocessing steps. We all share these steps.

In [4]:
# Drop unnecessary columns
columns_to_drop = ['Name', 'RescuerID', 'PetID', 'Description']
df = df.drop(columns=columns_to_drop, axis=1)  # 1 refers to columns

# Drop rows with missing data
df = df.dropna()

# Find z-scores for numerical columns
z_scores_fee = np.abs(stats.zscore(df['Fee']))
z_scores_age = np.abs(stats.zscore(df['Age']))

# Find outliers based on threshold of 5 std. deviations
outlier_rows_fee = z_scores_fee > 5
outlier_rows_age = z_scores_age > 5
combined_outlier_rows = outlier_rows_fee | outlier_rows_age

# Remove outliers
df = df[~combined_outlier_rows]

print(df)

       Type  Age  Breed1  Breed2  Gender  Color1  Color2  Color3  \
0         2    3     299       0       1       1       7       0   
1         2    1     265       0       1       1       2       0   
2         1    1     307       0       1       2       7       0   
3         1    4     307       0       2       1       2       0   
4         1    1     307       0       1       1       0       0   
...     ...  ...     ...     ...     ...     ...     ...     ...   
14988     2    2     266       0       3       1       0       0   
14989     2   60     265     264       3       1       4       7   
14990     2    2     265     266       3       5       6       7   
14991     2    9     266       0       2       4       7       0   
14992     1    1     307     307       1       2       0       0   

       MaturitySize  FurLength  Vaccinated  Dewormed  Sterilized  Health  \
0                 1          1           2         2           2       1   
1                 2          2 

# Neural Network Pre-Processing

This is pre-processing that is specific to my neural network model.

Scale numerical attributes with `StandardScaler`. According to the `scikit-learn` [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), `StandardScaler` should be used to scale numerical attributes for input to a machine learning estimator. This is to prevent features with large values from dominating the learning process.

In [5]:
scaler = StandardScaler()
features_to_scale = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt']
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
print(df[features_to_scale])

            Age  Quantity       Fee  VideoAmt  PhotoAmt
0     -0.430954 -0.393225  1.515422 -0.164405 -0.828418
1     -0.558874 -0.393225 -0.308343 -0.164405 -0.542567
2     -0.558874 -0.393225 -0.308343 -0.164405  0.886685
3     -0.366994 -0.393225  2.427305 -0.164405  1.172536
4     -0.558874 -0.393225 -0.308343 -0.164405 -0.256717
...         ...       ...       ...       ...       ...
14988 -0.494914  1.633300 -0.308343 -0.164405 -0.256717
14989  3.214782  0.282284 -0.308343 -0.164405 -0.256717
14990 -0.494914  2.308808  0.238787 -0.164405  0.314984
14991 -0.047192 -0.393225 -0.308343 -0.164405 -0.256717
14992 -0.558874 -0.393225 -0.308343 -0.164405 -0.828418

[14796 rows x 5 columns]


One-hot encode non-binary categorical features with `pd.get_dummies()`, documentation is [here](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html). The below columns have values [1, 2, 3] for "Yes", "No", and "Not Sure". The one-hot encoding is done to prevent the neural network from assuming 1 < 2 < 3 for these values.

In [6]:
features_to_encode = ['Type', 'Gender', 'Vaccinated', 'Dewormed', 'Sterilized']
df = pd.get_dummies(df, columns=features_to_encode, dtype=np.int8)  # Changed to int8 for manual implementation
print(df)

            Age  Breed1  Breed2  Color1  Color2  Color3  MaturitySize  \
0     -0.430954     299       0       1       7       0             1   
1     -0.558874     265       0       1       2       0             2   
2     -0.558874     307       0       2       7       0             2   
3     -0.366994     307       0       1       2       0             2   
4     -0.558874     307       0       1       0       0             2   
...         ...     ...     ...     ...     ...     ...           ...   
14988 -0.494914     266       0       1       0       0             2   
14989  3.214782     265     264       1       4       7             2   
14990 -0.494914     265     266       5       6       7             3   
14991 -0.047192     266       0       4       7       0             1   
14992 -0.558874     307     307       2       0       0             2   

       FurLength  Health  Quantity       Fee  State  VideoAmt  PhotoAmt  \
0              1       1 -0.393225  1.515422  41

Split `Breed1`, `Breed2`, and `State` features into N features each based on frequency of values.

The breed columns have 306 unique values that have been label encoded, [1, 2, ..., 306]. We don't want the neural network to think there's a relationship between the numbers, like 100 < 101, but we can't one-hot encode because we will have too many columns.

Instead, we create a column for each of the top 3 most common values, and another for "other".

I used ChatGPT to help me with the dataframe manipulation in this code block, as I don't often use `pandas`.

In [7]:
n = 3
features_to_split = ['Breed1', 'Breed2', 'State']

for feature in features_to_split:
    top_n_values = df[feature].value_counts().head(n)
    print(f'Top {n} values for {feature}:\n{top_n_values}\n')

    top_n_value_names = top_n_values.index
    for index, row in df.iterrows():
        # If value isn't top N frequency, replace with -1 (other)
        if row[feature] not in top_n_value_names:
            df.at[index, feature] = -1

    df = pd.get_dummies(df, columns=[feature], dtype=np.int8)

print(df)

Top 3 values for Breed1:
Breed1
307    5903
266    3623
265    1257
Name: count, dtype: int64

Top 3 values for Breed2:
Breed2
0      10613
307     1723
266      597
Name: count, dtype: int64

Top 3 values for State:
State
41326    8595
41401    3800
41327     830
Name: count, dtype: int64

            Age  Color1  Color2  Color3  MaturitySize  FurLength  Health  \
0     -0.430954       1       7       0             1          1       1   
1     -0.558874       1       2       0             2          2       1   
2     -0.558874       2       7       0             2          2       1   
3     -0.366994       1       2       0             2          1       1   
4     -0.558874       1       0       0             2          1       1   
...         ...     ...     ...     ...           ...        ...     ...   
14988 -0.494914       1       0       0             2          2       1   
14989  3.214782       1       4       7             2          2       1   
14990 -0.494914       5 

# Neural Network sklearn Implementation

Set up neural network testing and training data. 85% is training, 10% is validation, and 5% is test.

In [8]:
# Separate X and y on input data
target_feature = 'AdoptionSpeed'
features = df.drop(columns=[target_feature])
target = df[target_feature]

# Split training data, 15% is for testing & validation
x_train, x_test_val, y_train, y_test_val = train_test_split(
    features, target, test_size=0.15, shuffle=True, stratify=target, random_state=42
)

# 5% of data is testing, 10% is validation
x_test, x_val, y_test, y_val = train_test_split(
    x_test_val, y_test_val, test_size=1 / 3, shuffle=True, stratify=y_test_val, random_state=42
)

Fit the neural network and assess accuracy on training, validation, and test sets.

- `max-iter` was set to 500 to prevent the neural network from giving up due to too many iterations.

- `hidden_layer_sizes` was determined after experimenting with different parameters with `GridSearchCV`. I ran a grid search on my local machine with a variety of layer sizes and found that `(38, 5)` gave the highest accuracy on test and validation data. 38 corresponds to the number of input features, and 5 corresponds to the number of categories we're predicting.

- The `solver` of `adam` was kept because the `scikit-learn` [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) recommends `adam` for datasets with thousands of samples or more. I considered using the solver `lbfgs`, but the neural network with `lbfgs` never converged.

- `random_state` was set to 42 to make the predictions reproducible.

In [9]:
classifier = MLPClassifier(max_iter=500, hidden_layer_sizes=(38, 5), random_state=42)

# Fit to training data and make predictions on test data
print('Training neural network...')
classifier.fit(x_train, y_train)
test_predictions = classifier.predict(x_test)

accuracy = classifier.score(x_train, y_train)
print(f'\nAccuracy on training: {round(accuracy, 3) * 100}%')

accuracy = classifier.score(x_val, y_val)
print(f'Accuracy on validation: {round(accuracy, 3) * 100}%')

accuracy = classifier.score(x_test, y_test)
print(f'Accuracy on test: {round(accuracy, 3) * 100}%\n')

# zero_divison=0 returns 0 precision/recall if no positive samples for a class
avg_precision = precision_score(y_test, test_predictions, average='weighted', zero_division=0)
avg_recall = recall_score(y_test, test_predictions, average='weighted', zero_division=0)
precision_per_class = precision_score(y_test, test_predictions, average=None, zero_division=0)
recall_per_class = recall_score(y_test, test_predictions, average=None, zero_division=0)

print(f'Average weighted precision: {round(avg_precision, 3) * 100}%')
print(f'Average weighted recall: {round(avg_recall, 3) * 100}%\n')
for class_label, precision, recall in zip(range(len(precision_per_class)), precision_per_class, recall_per_class):
    print(f'Class {class_label}: Precision = {round(precision, 3) * 100}%, Recall = {round(recall, 3) * 100}%')
print()

unique_values, counts = np.unique(test_predictions, return_counts=True)
print(f'Counts of each class in prediction: {dict(zip(unique_values, counts))}')

Training neural network...

Accuracy on training: 45.0%
Accuracy on validation: 40.0%
Accuracy on test: 38.7%

Average weighted precision: 36.9%
Average weighted recall: 38.7%

Class 0: Precision = 0.0%, Recall = 0.0%
Class 1: Precision = 30.9%, Recall = 29.5%
Class 2: Precision = 33.7%, Recall = 43.1%
Class 3: Precision = 37.7%, Recall = 13.3%
Class 4: Precision = 47.4%, Recall = 64.9%

Counts of each class in prediction: {1: 291, 2: 510, 3: 114, 4: 565}


Perform k-fold cross validation for modified parameters.

Output cross-validation accuracy, precision, and recall (weighted averages).

In [10]:
num_folds = 10
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted', zero_division=0),
    'recall': make_scorer(recall_score, average='weighted', zero_division=0),
}
scores_modified = cross_validate(classifier, x_train, y_train, cv=num_folds, scoring=scoring_metrics)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores_modified["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores_modified["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores_modified["test_recall"].mean(), 3) * 100}%')


10-fold validation accuracy mean: 37.8%
10-fold validation precision mean: 35.9%
10-fold validation recall mean: 37.8%


Repeating the same experiment with a default instance of `MLPClassifier`. `max_iter` was changed to 500 because the default value would not converge.

In [11]:
classifier = MLPClassifier(max_iter=500, random_state=42)

# Fit to training data and make predictions on test data
print('Training neural network...')
classifier.fit(x_train, y_train)
test_predictions = classifier.predict(x_test)

accuracy = classifier.score(x_train, y_train)
print(f'\nAccuracy on training: {round(accuracy, 3) * 100}%')

accuracy = classifier.score(x_val, y_val)
print(f'Accuracy on validation: {round(accuracy, 3) * 100}%')

accuracy = classifier.score(x_test, y_test)
print(f'Accuracy on test: {round(accuracy, 3) * 100}%\n')

# zero_divison=0 returns 0 precision/recall if no positive samples for a class
avg_precision = precision_score(y_test, test_predictions, average='weighted', zero_division=0)
avg_recall = recall_score(y_test, test_predictions, average='weighted', zero_division=0)
precision_per_class = precision_score(y_test, test_predictions, average=None, zero_division=0)
recall_per_class = recall_score(y_test, test_predictions, average=None, zero_division=0)

print(f'Average weighted precision: {round(avg_precision, 3) * 100}%')
print(f'Average weighted recall: {round(avg_recall, 3) * 100}%\n')
for class_label, precision, recall in zip(range(len(precision_per_class)), precision_per_class, recall_per_class):
    print(f'Class {class_label}: Precision = {round(precision, 3) * 100}%, Recall = {round(recall, 3) * 100}%')
print()

unique_values, counts = np.unique(test_predictions, return_counts=True)
print(f'Counts of each class in prediction: {dict(zip(unique_values, counts))}')

Training neural network...

Accuracy on training: 51.2%
Accuracy on validation: 40.699999999999996%
Accuracy on test: 37.1%

Average weighted precision: 35.3%
Average weighted recall: 37.1%

Class 0: Precision = 0.0%, Recall = 0.0%
Class 1: Precision = 30.5%, Recall = 23.9%
Class 2: Precision = 33.0%, Recall = 38.6%
Class 3: Precision = 35.9%, Recall = 21.4%
Class 4: Precision = 44.0%, Recall = 61.3%

Counts of each class in prediction: {0: 8, 1: 239, 2: 466, 3: 192, 4: 575}


Perform k-fold cross validation for default parameters.

Output cross-validation accuracy, precision, and recall (weighted averages).

In [12]:
num_folds = 10
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted', zero_division=0),
    'recall': make_scorer(recall_score, average='weighted', zero_division=0),
}
scores_default = cross_validate(classifier, x_train, y_train, cv=num_folds, scoring=scoring_metrics)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores_default["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores_default["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores_default["test_recall"].mean(), 3) * 100}%')


10-fold validation accuracy mean: 36.199999999999996%
10-fold validation precision mean: 34.9%
10-fold validation recall mean: 36.199999999999996%


Find p-values for accuracy, precision, and recall from cross-validations.

Got help from Ashley for this code.

In [13]:
alpha = 0.05  # Significance level

ttest_accuracy, p_val_accuracy = ttest_rel(scores_modified['test_accuracy'], scores_default['test_accuracy'])
ttest_precision, p_val_precision = ttest_rel(scores_modified['test_precision'], scores_default['test_precision'])
ttest_recall, p_val_recall = ttest_rel(scores_modified['test_recall'], scores_default['test_recall'])

print(f'\nAccuracy p-val: {round(p_val_accuracy, 3)}')
print(f'Precision p-val: {round(p_val_precision, 3)}')
print(f'Recall p-val: {round(p_val_recall, 3)}')


Accuracy p-val: 0.0
Precision p-val: 0.015
Recall p-val: 0.0


# Neural Network Manual Implementation

Define the classes used for my manual implementation of sklearn's neural network:
- `SoftmaxLogLoss`: Combined Softmax activation layer and Log Loss function.
- `LogLoss`: Loss function for model, finds log loss for each sample and average loss.
- `Softmax`: Softmax activation layer for the model's output. Gives class probabilities for samples.
- `ReLU`: Rectified linear unit activation layer. Used as the activation function for hidden layers.
- `Layer`: Implements a hidden layer with neurons; each neuron has weights and a bias.
- `OptimizerSGD`: A simple implementation of stochastic gradient descent, optimizes the model's parameters.
- `NeuralNet`: Encapsulates all of the above classes with helper functions to mimimc sklearn's MLP class.

In [14]:
"""
Citations:

1) "Neural Networks from Scratch in Python" by Harrison Kinsley, Daniel Kukieła (2020), https://nnfs.io/

This textbook and the corresponding YouTube series were a great help in this project.
With permission from Manish, I used this book's solution for the .backward() methods (backpropagation), as
implementing these methods myself for each layer was beyond my understanding (especially the Softmax and 
Log Loss layers). I used chapters 8 through 10 in the book.

2) These playlists helped me understand how to implement the forward passes:

"Neural Networks from Scratch in Python" by sentdex:
https://youtube.com/playlist?list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3&feature=shared

"Neural networks" by 3Blue1Brown:
https://youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&feature=shared
"""


class SoftmaxLogLoss:
    """Combined Softmax activation function and Log Loss function.

    This is done to calculate the combined gradient of the loss function
    and the Softmax activation function. This ends up being much faster
    during backpropagation compared to separately calculating the gradient
    of the activation and loss functions.

    Attributes:
        activation (Softmax): Softmax activation layer, finds class probabilities.
        loss (LogLoss): Log loss layer, finds cross entropy loss for samples.
        dinputs (np.ndarray): Gradient of loss function w.r.t. this layer's inputs.
            - Used for backpropagation during training.
    """

    def __init__(self):
        self.activation = Softmax()
        self.loss = LogLoss()

    def forward(self, inputs: np.ndarray, target_vals: np.ndarray):
        """Execute forward pass for Softmax and Loss layers."""
        self.activation.forward(inputs)  # Output layer's Softmax activation
        self.loss.forward(self.activation.output, target_vals)  # Calculate loss for each sample

    def backward(self, dvalues: np.ndarray, target_vals: np.ndarray):
        """Find the gradient of the model's loss function w.r.t. this layer's inputs."""
        num_samples = len(dvalues)

        # If target labels are one-hot encoded, convert to [0, 1, 2, ...]
        if target_vals.ndim > 1:
            # Get index of max value in each row
            target_vals = np.argmax(target_vals, axis=1)

        self.dinputs = dvalues.copy()  # Don't modify input gradient
        self.dinputs[range(num_samples), target_vals] -= 1  # Calculate gradient
        self.dinputs = self.dinputs / num_samples  # Normalize gradient


class LogLoss:
    """Log loss (cross entropy) function for the neural network.

    This class calculates log loss for each sample between the model's predicted
    probabilities and target values. The class also calculates average loss
    across all samples.

    Input to this layer is the output of the Softmax layer, which is a matrix of
    predicted class probabilities for each sample.

    Attributes:
        log_losses (np.ndarray): Log losses for each sample.
        avg_loss (float): Average log loss across all samples.
        dinputs (np.ndarray): Gradient of loss function w.r.t. this layer's inputs.
            - Used for backpropagation during training.
    """

    def forward(self, predictions: np.ndarray, target_vals: np.ndarray):
        """Find log loss values for each sample, and average log loss across all samples."""
        # Target vals are a 1D vector of scalars, e.g. [0, 1, 2]
        # Treat each value as an index
        if target_vals.ndim == 1:
            target_probs = predictions[range(len(predictions)), target_vals]

        # Target values are one-hot encoded
        elif target_vals > 1:
            target_probs = predictions * target_vals  # Everything 0 except target index
            target_probs = np.sum(target_probs, axis=1)  # Isolates non-zero value

        # Ensure softmax vals are in range (0, 1) excluding 0 and 1
        # Prevents division by 0 when we compute average loss
        # -ln(1) = 0 and -ln(0) = inf
        probs_clipped = np.clip(target_probs, 1e-7, 1 - 1e-7)

        # Compute loss for each sample with negative log (log-loss)
        # Perfect prediction has all ones
        self.log_losses = -np.log(probs_clipped)

        # Compute average loss across all samples
        self.avg_loss = np.mean(self.log_losses)

    def backward(self, dvalues, y_true):
        """Find the gradient of the model's loss function w.r.t. this layer's inputs."""
        num_samples = len(dvalues)
        num_labels = len(dvalues[0])

        # If labels are a 1D vector, convert to one-hot encoding
        if y_true.ndim == 1:
            y_true = np.eye(num_labels)[y_true]

        self.dinputs = -y_true / dvalues  # Calculate gradient
        self.dinputs = self.dinputs / num_samples  # Normalize gradient


class Softmax:
    """Calculates a probability distribution of class predictions for each sample.

    The output of this class is provided to the log loss function at the model's output.

    Attributes:
        output (np.ndarray): Probability distribution for classes for every sample.
        inputs (np.ndarray): Inputs to the layer, kept for backpropagation.
        dinputs (np.ndarray): Gradient of loss function w.r.t. this layer's inputs.
            - Used for backpropagation during training.
    """

    def forward(self, inputs: np.ndarray):
        """Finds probabilities that each sample belongs to each class."""
        self.inputs = inputs  # Keep inputs for backpropagation

        # Find max of all features for each sample
        # Result has same number of rows as the input, one per sample
        max_of_each_row = np.max(inputs, axis=1, keepdims=True)

        # Subtract max value in each row, prevents overflow
        exp_inputs = np.exp(inputs - max_of_each_row)

        # Find sum of all features for each sample
        # Result has same number of rows as the input, one per sample
        sum_of_each_row = np.sum(exp_inputs, axis=1, keepdims=True)

        # Divide each row by its corresponding sum
        probabilities = exp_inputs / sum_of_each_row

        self.output = probabilities

    def backward(self, dvalues):
        """Find the gradient of the model's loss function w.r.t. this layer's inputs."""
        self.dinputs = np.empty_like(dvalues)  # Uninitialized array

        # Calculate Jacobian matrix of output and sample-wise gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


class ReLU:
    """The Rectified Linear Unit (ReLU) activation function.

    Applies the ReLU activation function to a layer's output.
    Used to make a neural network's prediction nonlinear.

    Attributes:
        output (np.ndarray): The result of applying the ReLU function to each element in a layer's output.
        inputs (np.ndarray): Inputs to the layer, kept for backpropagation.
    """

    def forward(self, inputs: np.ndarray):
        """Applies the ReLU function element-wise: the maximum of 0 and the element."""
        self.output = np.maximum(0, inputs)
        self.inputs = inputs  # Keep inputs for for backpropagation

    def backward(self, dvalues: np.ndarray):
        """Find the gradient of the model's loss function w.r.t. this layer's inputs."""
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


class Layer:
    """One hidden layer in a neural network.

    Attributes:
        weights (np.ndarray): An (n_inputs, n_neurons) matrix of feature weights per neuron.
            - Weights are shape (n_inputs, n_neurons) to enable the dot product.
            - Each column has the weights for one neuron, a row has the weights for one feature.
            - Multiplying the matrix by a constant keeps the weights small.
        biases (np.ndarray): A (1, n_neurons) vector of biases, one for each neuron.
        output (np.ndarray): A weighted sum + bias for each neuron in the layer.
        inputs (np.ndarray): Inputs to the layer, kept for backpropagation.
        dweights (np.ndarray): Gradient of loss function w.r.t. this layer's weights.
        dbiases (np.ndarray): Gradient of loss function w.r.t. this layer's biases.
        dinputs (np.ndarray): Gradient of loss function w.r.t. this layer's inputs.
    """

    def __init__(self, n_inputs: int, n_neurons: int):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs: np.ndarray):
        """Computes a weighted sum + bias for each neuron (the dot product)."""
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues: np.ndarray):
        """Find the gradient of the model's loss function w.r.t. this layer's weights, biases, & inputs."""
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)


class OptimizerSGD:
    """A stochastic gradient descent optimizer for the model's parameters.

    Adjusts the weights and biases of a layer using the negative gradient
    of the loss function w.r.t each weight and bias, scaled by a learning rate.

    Attributes:
        learning_rate (float): A coefficient that controls how large our "steps" are.
    """

    def __init__(self, learning_rate: float = 0.1):
        self.learning_rate = learning_rate

    def update_params(self, layer: Layer):
        """Update the weights and biases of a layer using the gradient of the loss function."""
        layer.weights += -(self.learning_rate * layer.dweights)
        layer.biases += -(self.learning_rate * layer.dbiases)


class NeuralNet:
    """A neural network that tries to minimize a log loss function using stochastic gradient descent.

    Models an input layer, two hidden layers, and an output layer.
    Hidden layers use the ReLU activation function, output uses Softmax.
    Loss is calculated using log loss (cross entropy).
    Loss is minimized using stochastic gradient descent.
    """

    def __init__(
        self,
        input_num_features: int,
        output_num_classes: int,
        hidden_layer_sizes: tuple[int, int],
        max_iter: int = 200,
        learning_rate: float = 0.1,
    ):
        self.layer1 = Layer(input_num_features, hidden_layer_sizes[0])  # Input and first hidden layer
        self.activation1 = ReLU()
        self.layer2 = Layer(hidden_layer_sizes[0], hidden_layer_sizes[1])  # Second hidden layer
        self.activation2 = ReLU()
        self.layer3 = Layer(hidden_layer_sizes[1], output_num_classes)  # Output layer
        self.loss_activation = SoftmaxLogLoss()
        self.sgd_optimizer = OptimizerSGD(learning_rate=learning_rate)
        self.max_iter = max_iter

    def _get_numpy_array(self, arr: np.ndarray | pd.DataFrame) -> np.ndarray:
        """Convert a Pandas dataframe/series to a Numpy array if the input is a dataframe/series."""
        arr_np = arr
        if isinstance(arr, pd.DataFrame) or isinstance(arr, pd.Series):
            arr_np = arr.to_numpy()
        return arr_np

    def _get_class_predictions(self) -> np.ndarray:
        """Return a list of class predictions for each sample, e.g. [0, 1, ..., 4]."""
        # Probabilities that each sample belongs to each class, from Softmax output
        class_probabilities = self.loss_activation.activation.output

        # Index of the highest probability class per sample
        # Represents what class was predicted for each sample, e.g. [0, 1, .., 4]
        class_predictions = np.argmax(class_probabilities, axis=1)

        return class_predictions

    def fit(self, x_train: np.ndarray | pd.DataFrame, y_train: np.ndarray | pd.DataFrame) -> np.ndarray:
        """Train the model on x, y data using S.G.D over max_iter epochs, return class predictions."""
        x_train_np = self._get_numpy_array(x_train)
        y_train_np = self._get_numpy_array(y_train)

        percent_iterations = self.max_iter / 5

        # Train max_iter number of times
        for epoch in range(self.max_iter):
            # Forward pass
            self.layer1.forward(x_train_np)
            self.activation1.forward(self.layer1.output)
            self.layer2.forward(self.activation1.output)
            self.activation2.forward(self.layer2.output)
            self.layer3.forward(self.activation2.output)
            self.loss_activation.forward(self.layer3.output, y_train_np)

            if epoch % percent_iterations == 0:
                print(f'Epoch: {epoch}; Loss: {self.loss_activation.loss.avg_loss}')

            # Backward pass
            self.loss_activation.backward(self.loss_activation.activation.output, y_train_np)
            self.layer3.backward(self.loss_activation.dinputs)
            self.activation2.backward(self.layer3.dinputs)
            self.layer2.backward(self.activation2.dinputs)
            self.activation1.backward(self.layer2.dinputs)
            self.layer1.backward(self.activation1.dinputs)

            # Tune model parameters
            self.sgd_optimizer.update_params(self.layer1)
            self.sgd_optimizer.update_params(self.layer2)
            self.sgd_optimizer.update_params(self.layer3)

        return self._get_class_predictions()

    def predict(self, x_test: np.ndarray | pd.DataFrame) -> np.ndarray:
        """Predicts classes for test data, returns the class predictions."""
        x_test_np = self._get_numpy_array(x_test)

        # Forward pass
        self.layer1.forward(x_test_np)
        self.activation1.forward(self.layer1.output)
        self.layer2.forward(self.activation1.output)
        self.activation2.forward(self.layer2.output)
        self.layer3.forward(self.activation2.output)
        self.loss_activation.activation.forward(self.layer3.output)  # Only probabilities, no loss

        return self._get_class_predictions()

Define a function to perform cross validation on our "from scratch" implementation.

`cross_validate_manual()` comes from Ashley Pang.

In [15]:
# This code comes from Ashley (apang024)
def cross_validate_manual(model: NeuralNet, x, y, cv: int, scoring: dict, random_seed=None) -> dict:
    """Performs cross-validation on our custom model objects, returns results."""
    accuracy_list = []
    precision_list = []
    recall_list = []
    np.random.seed(random_seed)

    # Per split
    for _ in range(cv):
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=1 / cv, random_state=random_seed)
        model.fit(x_train, y_train)
        predictions = model.predict(x_val)

        for metric in scoring:
            if metric == 'accuracy':
                _accuracy_score = accuracy_score(y_val, predictions)
                accuracy_list.append(_accuracy_score)
            elif metric == 'precision':
                _precision_score = precision_score(y_val, predictions, average='weighted', zero_division=0)
                precision_list.append(_precision_score)
            elif metric == 'recall':
                _recall_score = recall_score(y_val, predictions, average='weighted', zero_division=0)
                recall_list.append(_recall_score)

    accuracy_array = np.array(accuracy_list)
    precision_array = np.array(precision_list)
    recall_array = np.array(recall_list)

    result_dictionary = {
        'test_accuracy': accuracy_array,
        'test_precision': precision_array,
        'test_recall': recall_array,
    }

    return result_dictionary

Fit the neural network and assess accuracy on training and test sets. Use the same data that the sklearn model used.

Here we use the same parameters as our optimized sklearn model:

- `max-iter` is 500, though my manual implementation performs very poorly because of this.

- `hidden_layer_sizes` is `(37, 5)` to represent 37 input features and 5 output classes. This was mistakenly set to `(38, 5)` for the sklearn version because I thought there were 38 input features.

- `learning_rate` is `0.001` to match the default value used in our [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) model.

- `input_num_features` and `output_num_classes` are explicitly set to `37` and `5` for my implementation to work correctly.

In [16]:
mlp = NeuralNet(
    input_num_features=37, output_num_classes=5, hidden_layer_sizes=(37, 5), max_iter=500, learning_rate=0.001
)

# Fit to training data and make predictions on test data
print('Training neural network...')
train_predictions = mlp.fit(x_train, y_train)
val_predictions = mlp.predict(x_val)
test_predictions = mlp.predict(x_test)

train_accuracy = accuracy_score(train_predictions, y_train.to_numpy())
print(f'\nAccuracy on training: {round(train_accuracy, 3) * 100}%')

val_accuracy = accuracy_score(val_predictions, y_val.to_numpy())
print(f'Accuracy on validation: {round(accuracy, 3) * 100}%')

test_accuracy = accuracy_score(test_predictions, y_test.to_numpy())
print(f'Accuracy on test: {round(test_accuracy, 3) * 100}%\n')

# zero_divison=0 returns 0 precision/recall if no positive samples for a class
avg_precision = precision_score(y_test, test_predictions, average='weighted', zero_division=0)
avg_recall = recall_score(y_test, test_predictions, average='weighted', zero_division=0)
precision_per_class = precision_score(y_test, test_predictions, average=None, zero_division=0)
recall_per_class = recall_score(y_test, test_predictions, average=None, zero_division=0)

print(f'Average weighted precision: {round(avg_precision, 3) * 100}%')
print(f'Average weighted recall: {round(avg_recall, 3) * 100}%\n')
for class_label, precision, recall in zip(range(len(precision_per_class)), precision_per_class, recall_per_class):
    print(f'Class {class_label}: Precision = {round(precision, 3) * 100}%, Recall = {round(recall, 3) * 100}%')
print()

unique_values, counts = np.unique(train_predictions, return_counts=True)
print(f'Training predictions: {train_predictions}')
print(f'Counts of each class in prediction: {dict(zip(unique_values, counts))}')

unique_values, counts = np.unique(test_predictions, return_counts=True)
print(f'Test predictions: {test_predictions}')
print(f'Counts of each class in prediction: {dict(zip(unique_values, counts))}')

Training neural network...
Epoch: 0; Loss: 1.6094459496021978
Epoch: 100; Loss: 1.6053942918620308
Epoch: 200; Loss: 1.6014999618200427
Epoch: 300; Loss: 1.5977553265778615
Epoch: 400; Loss: 1.5941531917133942

Accuracy on training: 28.000000000000004%
Accuracy on validation: 37.1%
Accuracy on test: 27.900000000000002%

Average weighted precision: 7.8%
Average weighted recall: 27.900000000000002%

Class 0: Precision = 0.0%, Recall = 0.0%
Class 1: Precision = 0.0%, Recall = 0.0%
Class 2: Precision = 0.0%, Recall = 0.0%
Class 3: Precision = 0.0%, Recall = 0.0%
Class 4: Precision = 27.900000000000002%, Recall = 100.0%

Training predictions: [4 4 4 ... 4 4 4]
Counts of each class in prediction: {4: 12576}
Test predictions: [4 4 4 ... 4 4 4]
Counts of each class in prediction: {4: 1480}


Perform k-fold cross validation for manual implementation with modified parameters.

Output cross-validation accuracy, precision, and recall (weighted averages).

In [17]:
# Get cross-validation metrics for the manual implementation (baseline parameters)
num_folds = 10
scoring_metrics = ['accuracy', 'precision', 'recall']
print(f'Performing cross-validation with {num_folds} folds...')
scores_manual_baseline = cross_validate_manual(mlp, x_train, y_train, cv=num_folds, scoring=scoring_metrics)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores_manual_baseline["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores_manual_baseline["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores_manual_baseline["test_recall"].mean(), 3) * 100}%')


Performing cross-validation with 10 folds...
Epoch: 0; Loss: 1.5905632431809045
Epoch: 100; Loss: 1.587177316206561
Epoch: 200; Loss: 1.5839165707875962
Epoch: 300; Loss: 1.5807752211115402
Epoch: 400; Loss: 1.5777477981179
Epoch: 0; Loss: 1.5750861248949932
Epoch: 100; Loss: 1.572317279219323
Epoch: 200; Loss: 1.569645885846085
Epoch: 300; Loss: 1.567067628601712
Epoch: 400; Loss: 1.5645784177648425
Epoch: 0; Loss: 1.5623489296678466
Epoch: 100; Loss: 1.5600460457837537
Epoch: 200; Loss: 1.5578206083727568
Epoch: 300; Loss: 1.5556693362482588
Epoch: 400; Loss: 1.5535891125246155
Epoch: 0; Loss: 1.5513656411223282
Epoch: 100; Loss: 1.5494005866244824
Epoch: 200; Loss: 1.5474985374930537
Epoch: 300; Loss: 1.5456569228082795
Epoch: 400; Loss: 1.543873295864539
Epoch: 0; Loss: 1.542565326840721
Epoch: 100; Loss: 1.540915629668147
Epoch: 200; Loss: 1.5393163737089386
Epoch: 300; Loss: 1.5377655831786368
Epoch: 400; Loss: 1.5362613744731382
Epoch: 0; Loss: 1.534543015384287
Epoch: 100; Loss

Find p-values for accuracy, precision, and recall from cross-validations.

Got help from Ashley for this code.

In [28]:
alpha = 0.05  # Significance level

ttest_accuracy, p_val_accuracy = ttest_rel(scores_modified['test_accuracy'], scores_manual_baseline['test_accuracy'])
ttest_precision, p_val_precision = ttest_rel(
    scores_modified['test_precision'], scores_manual_baseline['test_precision']
)
ttest_recall, p_val_recall = ttest_rel(scores_modified['test_recall'], scores_manual_baseline['test_recall'])

print(f'\nAccuracy p-val: {p_val_accuracy}')
print(f'Precision p-val: {p_val_accuracy}')
print(f'Recall p-val: {p_val_accuracy}')



Accuracy p-val: 5.1066823670443025e-09
Precision p-val: 5.1066823670443025e-09
Recall p-val: 5.1066823670443025e-09


We now repeat the same experiment, but with different parameters that make my manual implementation perform better:

- `max-iter` is 10000. My epochs don't seem to take as long as sklearn's, and more iterations greatly help my model.

- `learning_rate` is `0.1` to help my manual implementation find a local minimum more quickly during stochastic gradient descent.

In [19]:
mlp = NeuralNet(
    input_num_features=37, output_num_classes=5, hidden_layer_sizes=(37, 5), max_iter=10000, learning_rate=0.1
)

# Fit to training data and make predictions on test data
print('Training neural network...')
train_predictions = mlp.fit(x_train, y_train)
val_predictions = mlp.predict(x_val)
test_predictions = mlp.predict(x_test)

train_accuracy = accuracy_score(train_predictions, y_train.to_numpy())
print(f'\nAccuracy on training: {round(train_accuracy, 3) * 100}%')

val_accuracy = accuracy_score(val_predictions, y_val.to_numpy())
print(f'Accuracy on validation: {round(accuracy, 3) * 100}%')

test_accuracy = accuracy_score(test_predictions, y_test.to_numpy())
print(f'Accuracy on test: {round(test_accuracy, 3) * 100}%\n')

# zero_divison=0 returns 0 precision/recall if no positive samples for a class
avg_precision = precision_score(y_test, test_predictions, average='weighted', zero_division=0)
avg_recall = recall_score(y_test, test_predictions, average='weighted', zero_division=0)
precision_per_class = precision_score(y_test, test_predictions, average=None, zero_division=0)
recall_per_class = recall_score(y_test, test_predictions, average=None, zero_division=0)

print(f'Average weighted precision: {round(avg_precision, 3) * 100}%')
print(f'Average weighted recall: {round(avg_recall, 3) * 100}%\n')
for class_label, precision, recall in zip(range(len(precision_per_class)), precision_per_class, recall_per_class):
    print(f'Class {class_label}: Precision = {round(precision, 3) * 100}%, Recall = {round(recall, 3) * 100}%')
print()

unique_values, counts = np.unique(train_predictions, return_counts=True)
print(f'Training predictions: {train_predictions}')
print(f'Counts of each class in prediction: {dict(zip(unique_values, counts))}')

unique_values, counts = np.unique(test_predictions, return_counts=True)
print(f'Test predictions: {test_predictions}')
print(f'Counts of each class in prediction: {dict(zip(unique_values, counts))}')

Training neural network...
Epoch: 0; Loss: 1.6094240230069676
Epoch: 2000; Loss: 1.463788552663659
Epoch: 4000; Loss: 1.381777374263621
Epoch: 6000; Loss: 1.3473923658904607
Epoch: 8000; Loss: 1.3305103163863892

Accuracy on training: 42.0%
Accuracy on validation: 37.1%
Accuracy on test: 39.300000000000004%

Average weighted precision: 38.6%
Average weighted recall: 39.300000000000004%

Class 0: Precision = 0.0%, Recall = 0.0%
Class 1: Precision = 33.900000000000006%, Recall = 39.300000000000004%
Class 2: Precision = 33.5%, Recall = 47.4%
Class 3: Precision = 38.2%, Recall = 13.0%
Class 4: Precision = 51.2%, Recall = 55.900000000000006%

Training predictions: [4 4 3 ... 1 4 3]
Counts of each class in prediction: {1: 1743, 2: 3414, 3: 1187, 4: 6232}
Test predictions: [4 1 2 ... 1 4 1]
Counts of each class in prediction: {1: 354, 2: 565, 3: 110, 4: 451}


Perform k-fold cross validation for manual implementation with modified parameters.

Output cross-validation accuracy, precision, and recall (weighted averages).

In [20]:
# Get cross-validation metrics for the manual implementation (modified parameters)
num_folds = 10
scoring_metrics = ['accuracy', 'precision', 'recall']
print(f'Performing cross-validation with {num_folds} folds...')
scores_manual_modified = cross_validate_manual(mlp, x_train, y_train, cv=num_folds, scoring=scoring_metrics)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores_manual_modified["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores_manual_modified["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores_manual_modified["test_recall"].mean(), 3) * 100}%')


Performing cross-validation with 10 folds...
Epoch: 0; Loss: 1.3124350664169302
Epoch: 2000; Loss: 1.296672883967074
Epoch: 4000; Loss: 1.2894313093995198
Epoch: 6000; Loss: 1.2759197359298349
Epoch: 8000; Loss: 1.2794160173653162
Epoch: 0; Loss: 1.2748584567959464
Epoch: 2000; Loss: 1.2578926791192646
Epoch: 4000; Loss: 1.2608271645901752
Epoch: 6000; Loss: 1.2541892221192297
Epoch: 8000; Loss: 1.2531472943839617
Epoch: 0; Loss: 1.2555860405503578
Epoch: 2000; Loss: 1.2605270730823566
Epoch: 4000; Loss: 1.2382423449477447
Epoch: 6000; Loss: 1.2391154465942367
Epoch: 8000; Loss: 1.2370342723064258
Epoch: 0; Loss: 1.2483579734204224
Epoch: 2000; Loss: 1.2348287560931186
Epoch: 4000; Loss: 1.2349045304484019
Epoch: 6000; Loss: 1.2278369306404964
Epoch: 8000; Loss: 1.2391186450808014
Epoch: 0; Loss: 1.236017399318491
Epoch: 2000; Loss: 1.2230810522429787
Epoch: 4000; Loss: 1.2201077796848903
Epoch: 6000; Loss: 1.2202236330842509
Epoch: 8000; Loss: 1.2183523318528693
Epoch: 0; Loss: 1.2318

Perform k-fold cross validation for the modified sklearn implementation.

Output cross-validation accuracy, precision, and recall (weighted averages).

In [24]:
# Get cross-validation metrics for "sklearn Modified" using our function
classifier = MLPClassifier(max_iter=500, hidden_layer_sizes=(38, 5), random_state=42)
num_folds = 10
scoring_metrics = ['accuracy', 'precision', 'recall']
print(f'Performing cross-validation with {num_folds} folds...')
scores_modified_custom = cross_validate_manual(classifier, x_train, y_train, cv=num_folds, scoring=scoring_metrics)

print(f'\n{num_folds}-fold validation accuracy mean: {round(scores_modified_custom["test_accuracy"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation precision mean: {round(scores_modified_custom["test_precision"].mean(), 3) * 100}%')
print(f'{num_folds}-fold validation recall mean: {round(scores_modified_custom["test_recall"].mean(), 3) * 100}%')


Performing cross-validation with 10 folds...

10-fold validation accuracy mean: 37.1%
10-fold validation precision mean: 35.6%
10-fold validation recall mean: 37.1%


Find p-values for accuracy, precision, and recall from cross-validations.

Got help from Ashley for this code.

In [25]:
alpha = 0.05  # Significance level

ttest_accuracy, p_val_accuracy = ttest_rel(
    scores_manual_modified['test_accuracy'], scores_modified_custom['test_accuracy']
)
ttest_precision, p_val_precision = ttest_rel(
    scores_manual_modified['test_precision'], scores_modified_custom['test_precision']
)
ttest_recall, p_val_recall = ttest_rel(scores_manual_modified['test_recall'], scores_modified_custom['test_recall'])

print(f'\nAccuracy p-val: {p_val_accuracy}')
print(f'Precision p-val: {p_val_accuracy}')
print(f'Recall p-val: {p_val_accuracy}')



Accuracy p-val: 0.00010913491703793282
Precision p-val: 0.00010913491703793282
Recall p-val: 0.00010913491703793282
